# <font size=35 color=lightgreen>** Sentiment API **<font>🥲
---

### <font size=12 color=lightgreen>Configuración Inicial (Librerías)</font>

#### 1. Procesamiento y Manipulación de Datos
* **`pandas`**
    * Nos ayuda con la manipulación y análisis de datos estructurados.
    * Carga el dataset (CSV), gestiona el DataFrame y permite filtrar o limpiar registros.
* **`numpy`**
    * Realiza las operaciones matemáticas y manejo de arrays eficientes.
    * Soporte numérico fundamental para las transformaciones vectoriales de los textos.

#### 2. Visualización y Análisis Exploratorio

* **`matplotlib.pyplot`**
    * Generación de gráficos estáticos.
    * Visualización básica de la distribución de clases (Positivo vs. Negativo).
* **`seaborn`**
    * Visualización de datos estadísticos avanzada.
    * Generación de matrices de confusión y gráficos de distribución estéticos para la presentación.

#### 3. Procesamiento de Lenguaje Natural (NLP) y Limpieza

* **`re`** (Regular Expressions)
    * Manejo de expresiones regulares.
    * Eliminación de ruido en el texto: URLs, menciones (@usuario), hashtags (#) y caracteres especiales no alfanuméricos.
* **`string`**
    * Constantes de cadenas comunes.
    * Provee listas estándar de signos de puntuación para su eliminación eficiente.

#### 4. Modelado y Machine Learning (Core)

* **`scikit-learn`**
    * Biblioteca principal de Machine Learning.
    * **`TfidfVectorizer`**: Transforma el texto limpio en vectores numéricos.
    * **`LogisticRegression`**: Algoritmo de clasificación supervisada.
    * **`metrics`**: Cálculo de precisión, recall y F1-score.
    * **`Pipeline`**: Encapsulamiento de los pasos de transformación y predicción.

#### 5. Persistencia e Integración
Herramientas para conectar el modelo con el Backend.

* **`joblib`**
    * Serialización eficiente de objetos Python.
    * Exportar (`dump`) el pipeline entrenado a un archivo `.joblib` y cargarlo (`load`) en la API para realizar predicciones.
* **`fastapi` & `uvicorn`**
    * Framework web moderno de alto rendimiento.
    * Exponer el modelo entrenado como un microservicio REST (endpoint `/predict`) para ser consumido por el Backend en Java.




---



### <font size=16  color=lightgreen> Importando librerías <font>



In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import re
import string
import chardet
import uvicorn
import sklearn
import fastapi
import joblib
import nltk
import unicodedata
import urllib.request
from io import StringIO
import urllib.response
import os
from pathlib import Path
from datetime import datetime
import warnings


### <font size = 8 color="lightgreen">Importación de los datasets<font>

#### **Función importación dataset**

In [111]:
def importar_dataset(url, sep=';'):
    """
    Importa dataset desde URL detectando encoding automáticamente.
    Acepta un argumento `sep` para especificar el separador del CSV (por defecto ';').
    """
    try:
        # 1. Descargar contenido una sola vez
        with urllib.request.urlopen(url) as response:
            content = response.read()

        # 2. Detectar encoding
        result = chardet.detect(content)
        encoding = result['encoding']
        print(f"🔍 Encoding detectado: {encoding} (confianza: {result['confidence']:.2%})")

        # 3. Decodificar y cargar en DataFrame
        decoded_content = content.decode(encoding, errors='replace')
        data = pd.read_csv(StringIO(decoded_content), sep=sep)

        print("✅ Archivo cargado correctamente")
        print(f"📊 Tamaño del dataset: {data.shape}")
        print("\n🔍 Muestra aleatoria (3 registros):")
        print(data.sample(3))

        return data

    except urllib.error.URLError as e:
        print(f"❌ Error de URL: {e}")
        return None
    except pd.errors.ParserError as e:
        print(f"❌ Error al parsear CSV: {e}")
        return None
    except Exception as e:
        print(f"❌ Error inesperado: {type(e).__name__}: {e}")
        return None

#### **Dataset1: sentimentdataset_es.csv**

In [112]:
df1_url_es = importar_dataset("https://github.com/ml-punto-tech/sentiment-api/raw/refs/heads/dev/data-science/datasets/datasets-origin/dataset1_esp.csv")


🔍 Encoding detectado: Windows-1252 (confianza: 72.97%)
✅ Archivo cargado correctamente
📊 Tamaño del dataset: (731, 15)

🔍 Muestra aleatoria (3 registros):
     Unnamed: 0.1  Unnamed: 0  \
525           527         531   
358           359         363   
14             14          14   

                                                  Text    Sentiment  \
525  En un concierto de U2, los acordes del himno d...      Neutral   
358  Apreciación por la vibrante cultura experiment...  Apreciación   
14   La tecnología está cambiando la forma en que v...      Neutral   

            Timestamp                User   Platform  \
525  15-05-2016 19:30  U2ConnectionSeeker  Instagram   
358  14-11-2016 13:45     CultureExplorer    Twitter   
14    19-01-2023 9:45      TechEnthusiast    Twitter   

                              Hashtags  Retweets  Likes      Country  Year  \
525                #Conexión #U2Anthem        25     50      Irlanda  2016   
358  #Apreciación #ExperienciaCultural        

#### **Dataset2: sentiment_analysis_dataset.csv**

In [113]:
df2_url_es = importar_dataset("https://github.com/ml-punto-tech/sentiment-api/raw/refs/heads/dev/data-science/datasets/datasets-origin/dataset2_esp.csv")

🔍 Encoding detectado: Windows-1252 (confianza: 73.00%)
✅ Archivo cargado correctamente
📊 Tamaño del dataset: (2540, 3)

🔍 Muestra aleatoria (3 registros):
                                                  texto  label sentimiento
1247  ¡Descubre el poder de María! ???? Conviértete ...      2    positivo
2157  La historia primero como mito, más tarde como ...      1     neutral
1124                             Creo que pediré pan xd      0    negativo


In [114]:
df3_url_es = importar_dataset("https://github.com/ml-punto-tech/sentiment-api/raw/refs/heads/dev/data-science/datasets/datasets-origin/dataset3_esp.csv", sep=',') # Probando con ',' como separador

🔍 Encoding detectado: Windows-1254 (confianza: 56.84%)
✅ Archivo cargado correctamente
📊 Tamaño del dataset: (740, 4)

🔍 Muestra aleatoria (3 registros):
       id plataforma sentimiento  \
248  9149     Nvidia     Neutral   
597  6713   Fortnite     Neutral   
253  9153     Nvidia     Neutral   

                                                 texto  
248  « Nvidia Shield no es mi dispositivo de uso di...  
597  @ClixHimself Todos usen el código Notclix en l...  
253  Ganar pruebas de benchmark es excelente para e...  


<font color='lightgreen' size=12>Filtrar datasets</font>

In [ ]:
# Función filtrar dataset
def filtrar_dataset(data):
    data_filtro = data[['texto', 'sentimiento']]
    data_filtro = data_filtro[data_filtro['texto'].str.strip() != ""]
    print(data_filtro.sample(3))
    return data_filtro

# Reemplazar nombre columnas Text por texto, Sentiment por sentimiento
df1_raw.rename({'Text':'texto', 'Sentiment':'sentimiento'}, axis=1, inplace=True)
df1_filtrado = filtrar_dataset(df1_raw)
df2_filtrado = filtrar_dataset(df2_raw)
df3_filtrado = filtrar_dataset(df3_raw)

                                                 texto  sentimiento
388  La euforia inunda cuando la última pieza del r...      Euforia
340  Surgen emociones agridulces al despedirse de u...      Neutral
322  Ojos temerosos que escanean las sombras, prisi...     Temeroso
592  Decidió aprender un nuevo instrumento.Día uno:...  Frustración
669  Intentando establecer un nuevo récord de más p...   Excitación
                                                  texto sentimiento
183   Es que estaba claro que en las fiestas DENI y ...    negativo
556                                 Frustrado y la csmr    negativo
755   viernes 23:58  me puse a jugar al solitario se...    negativo
1197  Que tiene como casi dos horas viéndose la espa...    positivo
1704  Quiero volver a ser el yo de antes, sin preocu...    positivo
                                                 texto sentimiento
430  Para quienes juegan CS:GO y sufren tirones, ac...     Neutral
587                  ¿Vale la pena reiniciar Fortn

### <font size= 12 color="lightgreen" >Explorando los datasets<font>

In [116]:
# Crear función para explorar datasets
def explorar_dataset(data):
    print('Filas: ' + str(data.shape[0]))
    print('Columnas: ' + str(data.shape[1]))
    print('\nColumnas: \n' + str(data.columns.tolist()))
    print('\nTipo de datos: \n' + str(data.dtypes))
    print('\nValores nulos: \n' + str(data.isnull().sum()))
    print('\nMuestra aleatoria (5 registros): \n' + str(data.sample(5)))

#### **Explorando Data1**

In [117]:
explorar_dataset(df1_filtrado)

Filas: 731
Columnas: 2

Columnas: 
['texto', 'sentimiento']

Tipo de datos: 
texto          object
sentimiento    object
dtype: object

Valores nulos: 
texto          0
sentimiento    0
dtype: int64

Muestra aleatoria (5 registros): 
                                                 texto sentimiento
198  El complejo rompecabezas de la vida me deja en...     Neutral
594  Cociné con éxito una comida gourmet para la fa...     Orgullo
295  Una escapada lúdica al carnaval de la vida, ri...    Positivo
696  Enfrentando el rechazo de la universidad de su...    Tristeza
188  Asco ante la corrupción que mancha a la sociedad.        Asco


#### **Explorando data2**

In [118]:
explorar_dataset(df1_filtrado)

Filas: 731
Columnas: 2

Columnas: 
['texto', 'sentimiento']

Tipo de datos: 
texto          object
sentimiento    object
dtype: object

Valores nulos: 
texto          0
sentimiento    0
dtype: int64

Muestra aleatoria (5 registros): 
                                                 texto    sentimiento
405  Saboreando el calor de una taza de chocolate e...        Neutral
193  El resentimiento se pudre, una herida que se n...  Resentimiento
278  Una lluvia compasiva, lágrimas de empatía caye...        Neutral
217  Pasar el día con un comportamiento indiferente...        Neutral
164  Abrumado por el dolor, extrañando profundament...          Dolor


In [119]:
explorar_dataset(df3_filtrado)

Filas: 740
Columnas: 2

Columnas: 
['texto', 'sentimiento']

Tipo de datos: 
texto          object
sentimiento    object
dtype: object

Valores nulos: 
texto          0
sentimiento    0
dtype: int64

Muestra aleatoria (5 registros): 
                                                 texto sentimiento
609  Apple acaba de lanzarse a la guerra nuclear co...     Neutral
384  Horario del servicio de mini transmisión: miér...     Neutral
739  Yo tendía a pensar que Ellison era un buen tip...     Neutral
218  Mac es una basura... dificultan el uso de tarj...     Neutral
599  @ Clixself Todos usan el código Notclix en una...     Neutral


### <font size=12 color=lightgreen>Limpiar textos</font>

#### **Función para limpieza de textos**

In [120]:
def limpiar_texto_sentimientos(texto):
    """
    Normaliza texto español preservando ñ y eliminando tildes.
    NO convierte a minúsculas para preservar intensidad emocional.
    """
    # Verifica si la entrada no es una cadena. Si no lo es, devuelve una cadena vacía.
    if not isinstance(texto, str):
        return ""

    # 1. Normaliza el texto para separar los caracteres base de sus diacríticos (ej., tildes).
    texto = unicodedata.normalize('NFD', texto)

    # 2. Reemplaza temporalmente las 'ñ' y 'Ñ' con marcadores especiales para preservarlas
    # durante la eliminación de diacríticos.
    texto = texto.replace('n\u0303', '@@@N_TILDE@@@')
    texto = texto.replace('ñ', '@@@N_TILDE@@@')
    texto = texto.replace('N\u0303', '@@@N_TILDE_MAYUS@@@')
    texto = texto.replace('Ñ', '@@@N_TILDE_MAYUS@@@')

    # 3. Elimina los caracteres diacríticos (como las tildes) del texto.
    texto = ''.join(
        char for char in texto
        if not unicodedata.combining(char)
    )

    # Restaura las 'ñ' y 'Ñ' utilizando los marcadores temporales.
    texto = texto.replace('@@@N_TILDE@@@', 'ñ')
    texto = texto.replace('@@@N_TILDE_MAYUS@@@', 'Ñ')

    # Variable para almacenar el resultado de la limpieza.
    resultado = texto
    chars = []

    # Itera sobre cada caracter en el resultado y añade solo los caracteres imprimibles a una lista.
    # Los caracteres no imprimibles (como los de control) son reemplazados por un espacio.
    for char in resultado:
        if char.isprintable():
            chars.append(char)
        else:
            chars.append(' ')
    resultado = ''.join(chars)

    # Elimina URLs que terminan en "..." (posibles URLs rotas).
    resultado = re.sub(r'https?://[^\s]*\.\.\.', '[URL_ROTA]', resultado)
    resultado = re.sub(r'www\.[^\s]*\\.\\.\\.', '[URL_ROTA]', resultado)

    # Normaliza los espacios múltiples a uno solo y elimina espacios al inicio y final.
    resultado = ' '.join(resultado.split())
    resultado = resultado.strip()


    # Devuelve el texto preprocesado.
    return resultado


#### **Análisis proceso de limpieza de textos**

In [121]:
def analizar_limpieza_sentimientos(df_antes, df_despues, nombre):
    """
    Análisis específico para tu función limpiar_texto_para_sentimientos
    """
    print(f"\n🔍 ANÁLISIS ESPECÍFICO: {nombre}")
    print("="*60)

    # 1. Cambios en caracteres específicos del español
    cambios_especificos = {
        'tildes_eliminadas': 0,
        'ñ_preservadas': 0,
        'urls_eliminadas': 0,
        'mayusculas_preservadas': 0
    }

    # Muestra de 50 textos para análisis detallado
    muestra = min(50, len(df_antes))

    for i in range(muestra):
        if i < len(df_despues):
            texto_antes = str(df_antes.iloc[i]['texto'])
            texto_despues = str(df_despues.iloc[i]['texto'])

            # Contar ñ preservadas
            if 'ñ' in texto_antes.lower() and 'ñ' in texto_despues.lower():
                cambios_especificos['ñ_preservadas'] += 1

            # Contar URLs eliminadas
            import re
            urls_antes = len(re.findall(r'https?://\S+', texto_antes))
            urls_despues = len(re.findall(r'https?://\S+', texto_despues))
            if urls_antes > urls_despues:
                cambios_especificos['urls_eliminadas'] += (urls_antes - urls_despues)

            # Verificar mayúsculas preservadas
            mayus_antes = sum(1 for c in texto_antes if c.isupper())
            mayus_despues = sum(1 for c in texto_despues if c.isupper())
            if mayus_antes > 0 and mayus_despues > 0:
                cambios_especificos['mayusculas_preservadas'] += 1

    print("📊 Cambios específicos de tu limpiador:")
    for cambio, cantidad in cambios_especificos.items():
        print(f"   • {cambio.replace('_', ' ').title()}: {cantidad} de {muestra} textos")



    print("="*60)

In [122]:
# Lista de dataframes para procesar
dataframes = [
    (df1_filtrado, "Dataset 1"),
    (df2_filtrado, "Dataset 2"),
    (df3_filtrado, "Dataset 3")
]

resultados = {}

for df, nombre in dataframes:
    # Aplicar limpieza
    df['Texto_Limpio'] = df['texto'].apply(limpiar_texto_sentimientos)

    # Guardar copia limpia
    resultados[nombre] = df.copy()

    # Mostrar info
    print(f"\n📁 {nombre}")
    print(f"   Registros: {len(df):,}")
    print(f"   Muestra (3 textos):")
    print(df[['texto', 'Texto_Limpio']].sample(3))

# Asignar a variables originales
df1_clean = resultados["Dataset 1"]
df2_clean = resultados["Dataset 2"]
df3_clean = resultados["Dataset 3"]

analizar_limpieza_sentimientos(df1_filtrado, df1_clean, "Dataset 1")
analizar_limpieza_sentimientos(df2_filtrado, df2_clean, "Dataset 2")
analizar_limpieza_sentimientos(df3_filtrado, df3_clean, "Dataset 3")


📁 Dataset 1
   Registros: 731
   Muestra (3 textos):
                                                 texto  \
398  Montar la adrenalina en los giros salvajes de ...   
320  La soledad, una luna solitaria en un cielo sin...   
577  Mientras las olas rompen contra la orilla, el ...   

                                          Texto_Limpio  
398  Montar la adrenalina en los giros salvajes de ...  
320  La soledad, una luna solitaria en un cielo sin...  
577  Mientras las olas rompen contra la orilla, el ...  

📁 Dataset 2
   Registros: 2,540
   Muestra (3 textos):
                                                  texto  \
630                                           impotente   
440   «Nos hiciste, Señor, para ti, y nuestro corazó...   
1277  Mk ahora si me mareo con una copa y el sereno ...   

                                           Texto_Limpio  
630                                           impotente  
440   «Nos hiciste, Señor, para ti, y nuestro corazo...  
1277  Mk ahora si 

In [123]:
df1_clean.sample(3)

,texto,sentimiento,Texto_Limpio
353,Sentirse bendecido por la comunidad solidaria ...,Positivo,Sentirse bendecido por la comunidad solidaria ...
224,Una determinación ardiente arde en su interior...,Positivo,Una determinacion ardiente arde en su interior...
384,Susurros de inspiración provenientes del susur...,Neutral,Susurros de inspiracion provenientes del susur...


In [124]:
df2_clean.sample(3)

,texto,sentimiento,Texto_Limpio
1806,también tengo qe aceptar q ya no es mi lugar s...,neutral,tambien tengo qe aceptar q ya no es mi lugar s...
384,Algo que me parece increíble y muy molesto de ...,negativo,Algo que me parece increible y muy molesto de ...
1337,Lo de Óscar Puente trayendo el AVE a Vigo la n...,positivo,Lo de Oscar Puente trayendo el AVE a Vigo la n...


In [125]:
df3_clean.sample(3)

,texto,sentimiento,Texto_Limpio
549,Bip bip . . store.playstation.com/,Neutral,Bip bip . . store.playstation.com/
668,Microsoft News afirma que se están preparando ...,Neutral,Microsoft News afirma que se estan preparando ...
295,GPU NVIDIA GeForce RTX 3090 y RTX 3080 Creator...,Neutral,GPU NVIDIA GeForce RTX 3090 y RTX 3080 Creator...


### <font size=12 color=lightgreen>Categorizar de sentimientos </font>

#### **Categorías Sentimientos**

In [126]:
# 1. Definimos las listas de sentimientos según su categoría
# Ver todos los sentimientos únicos para saber qué agrupar y ordenar alfabetizamente
sentimientos_unicos = sorted(df1_clean['sentimiento'].unique())
print(f"Total de sentimientos únicos: {len(sentimientos_unicos)}")
print(sentimientos_unicos)

# 2. SENTIMIENTOS POSITIVOS COMPLETOS (Bienestar, éxito, alegría, admiración)
positivos = [
    'Aceptación', 'Admiración', 'Adoración', 'Agradecido', 'Alegría', 'Amabilidad', 'Amor', 'Amistad', 'Apreciación', 'Armonía', 'Asombro', 'Cautivación', 'Celebración', 'Colorido', 'Confiado','Confianza', 'Contentamiento', 'Creatividad', 'Cumplimiento', 'Descubrimiento', 'Deslumbrar', 'Determinación', 'Disfrute','Diversión', 'Elegancia', 'Emoción', 'Empático', 'Empoderamiento',
    'Encantamiento', 'Energía', 'Entusiasmo', 'Esperanza', 'Euforia', 'Excitación', 'Felicidad', 'Grandeza', 'Gratitud', 'Inspiración', 'Inspirado', 'Intimidación', 'Juguetón', 'Logro','Maravilla', 'Melódico', 'Motivación', 'Optimismo', 'Orgullo',
    'Positividad', 'Positivo', 'Reconfortante', 'Resiliencia', 'Resplandor', 'Reverencia', 'Romance', 'Satisfacción', 'Serenidad','Ternura', 'Triunfo', 'Ánimo', 'Éxito','Elación','Éxtasis']
print(f'Sentimientos positivos: {len(positivos)}'),

# 3. SENTIMIENTOS NEGATIVOS COMPLETOS (Dolor, ira, miedo, estrés, pérdida)
negativos = [
    'Abrumado', 'Aburrimiento', 'Aislamiento', 'Amargura', 'Angustia', 'Anhelo', 'Ansiedad', 'Aprensivo', 'Arrepentimiento', 'Asco',  'Decepción', 'Desamor', 'Desesperación', 'Despectivo', 'Devastado',
    'Dolor', 'Enojo', 'Entumecimiento', 'Envidia', 'Envidioso', 'Frustración', 'Frustrado', 'Lástima', 'Obstáculo', 'Malo', 'Melancolía', 'Miedo', 'Negativo', 'Odiar', 'Pena', 'Pérdida', 'Reflexión', 'Resentimiento', 'Soledad', 'Sufrimiento', 'Temeroso', 'Traición', 'Tristeza', 'Verguenza']
print(f'Sentimientos negativos: {len(negativos)}')

# 4. SENTIMIENTOS NEUTRALES (Estados ambiguos o contemplativos)
neutros = ['Ambivalencia', 'Curiosidad', 'Neutral','Sorpresa','Anticipación']
print(f'Sentimientos neutros: {len(neutros)}')

categorias = [positivos, negativos, neutros]

# Verificación del total
total_clasificados = len(positivos) + len(negativos) + len(neutros)
print(f'\n✅ Total clasificado: {total_clasificados}/106 sentimientos')
print(f'   - Positivos: {len(positivos)} ({len(positivos)/106*100:.1f}%)')
print(f'   - Negativos: {len(negativos)} ({len(negativos)/106*100:.1f}%)')
print(f'   - Neutros: {len(neutros)} ({len(neutros)/106*100:.1f}%)')
print(f'Total: {len(positivos) + len(negativos) + len(neutros)}')
print()

# Verificar si existen elementos en las listas que no se encuentran en la lista sentimientos_unicos
for sentimiento in negativos + positivos + neutros:
    if sentimiento not in sentimientos_unicos:
        print(f"❌ Sentimiento no encontrado en el dataset: {sentimiento}")

# Verificar si todos los sentimientos del dataset están clasificados
for sentimiento in sentimientos_unicos:
    if sentimiento not in positivos + negativos + neutros:
        print(f"❌ Sentimiento no clasificado: {sentimiento}")
else:
    print("✅ Todos los sentimientos del dataset están clasificados.")


Total de sentimientos únicos: 104
['Abrumado', 'Aburrimiento', 'Aceptación', 'Admiración', 'Adoración', 'Agradecido', 'Aislamiento', 'Alegría', 'Amabilidad', 'Amargura', 'Ambivalencia', 'Amistad', 'Amor', 'Angustia', 'Anhelo', 'Ansiedad', 'Anticipación', 'Apreciación', 'Aprensivo', 'Armonía', 'Arrepentimiento', 'Asco', 'Asombro', 'Cautivación', 'Celebración', 'Colorido', 'Confiado', 'Confianza', 'Contentamiento', 'Creatividad', 'Cumplimiento', 'Curiosidad', 'Decepción', 'Desamor', 'Descubrimiento', 'Desesperación', 'Deslumbrar', 'Despectivo', 'Determinación', 'Devastado', 'Disfrute', 'Diversión', 'Dolor', 'Elegancia', 'Emoción', 'Empoderamiento', 'Empático', 'Encantamiento', 'Energía', 'Enojo', 'Entumecimiento', 'Entusiasmo', 'Envidia', 'Envidioso', 'Esperanza', 'Euforia', 'Excitación', 'Felicidad', 'Frustración', 'Frustrado', 'Grandeza', 'Gratitud', 'Inspiración', 'Inspirado', 'Intimidación', 'Juguetón', 'Logro', 'Lástima', 'Malo', 'Maravilla', 'Melancolía', 'Melódico', 'Miedo', 'Moti

#### **Función para categorizar sentimientos**

In [127]:
def categorizar_sentimiento(sentimiento, categorias):
    """
    Categoriza sentimientos solo si están en las listas definidas.
    Devuelve None para sentimientos no clasificados.
    """
    sent = str(sentimiento).strip().title()

    if sent in positivos:
        return 'positivo'
    elif sent in negativos:
        return 'negativo'
    elif sent in neutros:
        return 'neutral'
    else:
        # Devolvemos None para posterior filtrado
        return None


In [128]:
df1_clean.sample(3)

,texto,sentimiento,Texto_Limpio
279,"Nostalgia, un baile agridulce en el salón de b...",Orgullo,"Nostalgia, un baile agridulce en el salon de b..."
148,Reverencia por el significado histórico de un ...,Reverencia,Reverencia por el significado historico de un ...
237,"Perdido en el laberinto de pensamientos, la co...",Neutral,"Perdido en el laberinto de pensamientos, la co..."


In [129]:
df2_clean.sample(3)

,texto,sentimiento,Texto_Limpio
448,"Por primera vez, indagaciones rondan dentro de...",negativo,"Por primera vez, indagaciones rondan dentro de..."
1447,"Me acabo de subir al carro de mi hermano, me s...",positivo,"Me acabo de subir al carro de mi hermano, me s..."
2496,Despreocupado de lo que está pasando yo sigo s...,positivo,Despreocupado de lo que esta pasando yo sigo s...


In [130]:
df3_clean.sample(3)

,texto,sentimiento,Texto_Limpio
681,"¡O publica mi tutorial ""Aprende lo básico con ...",Neutral,"¡O publica mi tutorial ""Aprende lo basico con ..."
616,¿Es ella HULK?... store.playstation.com /,Neutral,¿Es ella HULK?... store.playstation.com /
266,La valoraciÃ³n de Nvidia se dispara en compara...,Neutral,La valoraciA³n de Nvidia se dispara en compara...


#### **Categorizar sentimientos**

In [131]:
df1_clean['Sentimiento_Final'] = df1_clean['sentimiento'].apply(
    lambda x: categorizar_sentimiento(x,categorias)
)

df1_categorized = df1_clean[df1_clean['Sentimiento_Final'].notna()].copy()

df2_clean['Sentimiento_Final'] = df2_clean['sentimiento'].apply(
    lambda x: categorizar_sentimiento(x,categorias)
)

df2_categorized = df2_clean[df2_clean['Sentimiento_Final'].notna()].copy()

df3_clean['Sentimiento_Final'] = df3_clean['sentimiento'].apply(
    lambda x: categorizar_sentimiento(x,categorias)
)

df3_categorized = df3_clean[df3_clean['Sentimiento_Final'].notna()].copy()

print(f"✅ df1: {len(df1_categorized)} registros categorizados")
print(f"✅ df2: {len(df2_categorized)} registros categorizados")
print(f"✅ df3: {len(df3_categorized)} registros categorizados")

✅ df1: 731 registros categorizados
✅ df2: 2540 registros categorizados
✅ df3: 740 registros categorizados


In [132]:
df1_clean.sample(3)

,texto,sentimiento,Texto_Limpio,Sentimiento_Final
680,Entré accidentalmente al salón de clases equiv...,Verguenza,Entre accidentalmente al salon de clases equiv...,negativo
379,La armonía resuena cuando los músicos tocan un...,Armonía,La armonia resuena cuando los musicos tocan un...,positivo
419,Inundado de serenidad mientras el sol se pone ...,Serenidad,Inundado de serenidad mientras el sol se pone ...,positivo


In [133]:
df2_clean.sample(3)

,texto,sentimiento,Texto_Limpio,Sentimiento_Final
1053,"Ah, ojalá poder guardar el día de hoy en un lu...",negativo,"Ah, ojala poder guardar el dia de hoy en un lu...",negativo
2369,Choque múltiple 3 autos en costanera en brilla...,positivo,Choque multiple 3 autos en costanera en brilla...,positivo
2139,Trincado de haka e incrédulo,neutral,Trincado de haka e incredulo,neutral


In [134]:
df3_clean.sample(3)

,texto,sentimiento,Texto_Limpio,Sentimiento_Final
598,@ClixHimself Todos usan el código Notclix en l...,Neutral,@ClixHimself Todos usan el codigo Notclix en l...,neutral
721,Threatpost | Controlamos el infierno en el ata...,Neutral,Threatpost | Controlamos el infierno en el ata...,neutral
568,Menos de 200,Neutral,Menos de 200,neutral


### <font color=lightgreen size=12>Limpiar dataset unificado</font>

#### **Función limpieza dataset unificado**

In [135]:
def limpiar_dataset_unificado(data, verbose=True):
    """
    Limpia dataset unificado para análisis de sentimientos.

    Proceso:
    1. Identifica y elimina CONTRADICCIONES (textos con diferentes sentimientos)
    2. Elimina DUPLICADOS exactos (mismo texto, mismo sentimiento)
    3. Limpieza final (espacios vacíos, NaN)

    Args:
        data: DataFrame con 'Texto_Limpio' y 'Sentimiento_Final'
        verbose: Si True, muestra análisis detallado

    Returns:
        DataFrame limpio, sin duplicados ni contradicciones
    """

    if verbose:
        print("🧹 LIMPIANDO DATASET UNIFICADO")
        print("-" * 50)
        print(f"Registros iniciales: {len(data):,}")
        print(f"Textos únicos iniciales: {data['Texto_Limpio'].nunique():,}")

    # Hacer copia para no modificar original
    df = data.copy()

    # ===== 1. ELIMINAR CONTRADICCIONES (PRIMERO) =====
    if verbose:
        print(f"\n1. 🔍 BUSCANDO CONTRADICCIONES...")

    # Textos con más de un sentimiento diferente
    conteo_sentimientos = df.groupby('Texto_Limpio')['Sentimiento_Final'].nunique()
    textos_con_contradiccion = conteo_sentimientos[conteo_sentimientos > 1].index.tolist()

    if textos_con_contradiccion:
        if verbose:
            print(f"   ⚠️  Encontradas: {len(textos_con_contradiccion):,} contradicciones")

            # Mostrar algunos ejemplos
            print(f"   • Ejemplos (primeros 2):")
            for texto in textos_con_contradiccion[:2]:
                sentimientos = df[df['Texto_Limpio'] == texto]['Sentimiento_Final'].unique()
                texto_corto = texto[:60] + "..." if len(texto) > 60 else texto
                print(f"     - '{texto_corto}'")
                print(f"       → Sentimientos: {', '.join(sentimientos)}")

        # Eliminar TODOS los registros de textos contradictorios
        df_sin_contradicciones = df[~df['Texto_Limpio'].isin(textos_con_contradiccion)].copy()

        if verbose:
            eliminados = len(df) - len(df_sin_contradicciones)
            print(f"   🗑️  Eliminados: {eliminados:,} registros por contradicciones")
    else:
        if verbose:
            print(f"   ✅ No hay contradicciones")
        df_sin_contradicciones = df.copy()

    # ===== 2. ELIMINAR DUPLICADOS EXACTOS =====
    if verbose:
        print(f"\n2. 🔍 BUSCANDO DUPLICADOS EXACTOS...")

    # Contar duplicados exactos (mismo texto, mismo sentimiento)
    conteo_duplicados = df_sin_contradicciones['Texto_Limpio'].value_counts()
    textos_duplicados = conteo_duplicados[conteo_duplicados > 1].index.tolist()

    if textos_duplicados:
        if verbose:
            print(f"   ⚠️  Encontrados: {len(textos_duplicados):,} textos duplicados")

            # Calcular cuántos registros se eliminarán
            total_a_eliminar = sum([conteo_duplicados[t] - 1 for t in textos_duplicados])
            print(f"   • Registros a eliminar: {total_a_eliminar:,}")

        # Eliminar duplicados (mantener primera aparición)
        df_sin_duplicados = df_sin_contradicciones.drop_duplicates(
            subset=['Texto_Limpio'],
            keep='first'
        )

        if verbose:
            eliminados = len(df_sin_contradicciones) - len(df_sin_duplicados)
            print(f"   🗑️  Eliminados: {eliminados:,} registros duplicados")
    else:
        if verbose:
            print(f"   ✅ No hay duplicados exactos")
        df_sin_duplicados = df_sin_contradicciones.copy()

    # ===== 3. LIMPIEZA FINAL =====
    if verbose:
        print(f"\n3. 🧹 LIMPIEZA FINAL...")

    df_final = df_sin_duplicados.copy()

    # Filtrar solo columnas necesarias
    df_final = df_final[['Texto_Limpio', 'Sentimiento_Final']]

    # Eliminar textos vacíos o solo espacios
    textos_vacios_antes = len(df_final)
    df_final = df_final[df_final['Texto_Limpio'].str.strip() != ""]
    textos_vacios_eliminados = textos_vacios_antes - len(df_final)

    if verbose and textos_vacios_eliminados > 0:
        print(f"   • Textos vacíos eliminados: {textos_vacios_eliminados}")

    # Eliminar sentimientos NaN
    sentimientos_nan_antes = len(df_final)
    df_final = df_final[df_final['Sentimiento_Final'].notna()]
    sentimientos_nan_eliminados = sentimientos_nan_antes - len(df_final)

    if verbose and sentimientos_nan_eliminados > 0:
        print(f"   • Sentimientos NaN eliminados: {sentimientos_nan_eliminados}")

    # ===== 4. VERIFICACIÓN Y RESUMEN =====
    if verbose:
        print(f"\n4. ✅ VERIFICACIÓN FINAL")
        print(f"   • Registros finales: {len(df_final):,}")
        print(f"   • Textos únicos finales: {df_final['Texto_Limpio'].nunique():,}")

        # Verificar que cada texto aparece solo una vez
        if len(df_final) == df_final['Texto_Limpio'].nunique():
            print(f"   🎯 ¡Dataset 100% limpio! Cada texto aparece solo una vez")
        else:
            diferencia = len(df_final) - df_final['Texto_Limpio'].nunique()
            print(f"   ⚠️  ¡Problema! Hay {diferencia} duplicados")

        # Resumen
        print(f"\n" + "=" * 50)
        print("📊 RESUMEN DE LIMPIEZA")
        print("=" * 50)

        total_eliminados = (len(data) - len(df_final))
        porcentaje_eliminado = (total_eliminados / len(data)) * 100

        print(f"Registros iniciales: {len(data):,}")
        print(f"Registros finales: {len(df_final):,}")
        print(f"Total eliminados: {total_eliminados:,} ({porcentaje_eliminado:.1f}%)")

        # Distribución de sentimientos
        print(f"\n📈 DISTRIBUCIÓN FINAL DE SENTIMIENTOS:")
        distribucion = df_final['Sentimiento_Final'].value_counts()
        for sentimiento, count in distribucion.items():
            porcentaje = (count / len(df_final)) * 100
            print(f"   • {sentimiento}: {count:,} ({porcentaje:.1f}%)")

    return df_final


In [136]:
df1_categorized.sample(3)
df2_categorized.sample(3)
df3_categorized.sample(3)

,texto,sentimiento,Texto_Limpio,Sentimiento_Final
325,Creo que voy a perder el tiempo con esa transm...,Neutral,Creo que voy a perder el tiempo con esa transm...,neutral
649,Microsoft Surface Studio 2 es un dispositivo e...,Neutral,Microsoft Surface Studio 2 es un dispositivo e...,neutral
602,@ClixHimself Todos usen su Notclix para ir a l...,Neutral,@ClixHimself Todos usen su Notclix para ir a l...,neutral


#### **Unificar datataset y limpieza**

In [137]:
print("=" * 70)
print("🔗 UNIFICANDO DATASETS CATEGORIZADOS")
print("=" * 70)

# Unificar los datasets categorizados
# implementacion con dos datasets
df_unificado = pd.concat([df1_categorized[['Texto_Limpio', 'Sentimiento_Final']], df2_categorized[['Texto_Limpio', 'Sentimiento_Final']]], ignore_index=True)
#implementacion con los tres datasets
#df_unificado = pd.concat([df1_categorized[['Texto_Limpio', 'Sentimiento_Final']], df2_categorized[['Texto_Limpio', 'Sentimiento_Final']], df3_categorized[['Texto_Limpio', 'Sentimiento_Final']]], ignore_index=True)

print(f"📦 Dataset unificado: {df_unificado.shape}")
print(f"   • Registros: {len(df_unificado):,}")
print(f"   • Textos únicos: {df_unificado['Texto_Limpio'].nunique():,}")


# %%
print("\n" + "=" * 70)
print("🧹 APLICANDO LIMPIEZA AL DATASET UNIFICADO")
print("=" * 70)

# Aplicar limpieza
df_final = limpiar_dataset_unificado(df_unificado, verbose=True)

🔗 UNIFICANDO DATASETS CATEGORIZADOS
📦 Dataset unificado: (3271, 2)
   • Registros: 3,271
   • Textos únicos: 2,848

🧹 APLICANDO LIMPIEZA AL DATASET UNIFICADO
🧹 LIMPIANDO DATASET UNIFICADO
--------------------------------------------------
Registros iniciales: 3,271
Textos únicos iniciales: 2,848

1. 🔍 BUSCANDO CONTRADICCIONES...
   ⚠️  Encontradas: 90 contradicciones
   • Ejemplos (primeros 2):
     - '"De manera apacible, se puede sacudir el mundo" MG'
       → Sentimientos: negativo, positivo
     - '"He aprendido que el valor no es la ausencia de miedo, sino ...'
       → Sentimientos: neutral, positivo
   🗑️  Eliminados: 212 registros por contradicciones

2. 🔍 BUSCANDO DUPLICADOS EXACTOS...
   ⚠️  Encontrados: 252 textos duplicados
   • Registros a eliminar: 301
   🗑️  Eliminados: 301 registros duplicados

3. 🧹 LIMPIEZA FINAL...

4. ✅ VERIFICACIÓN FINAL
   • Registros finales: 2,758
   • Textos únicos finales: 2,758
   🎯 ¡Dataset 100% limpio! Cada texto aparece solo una vez

📊 RESU

In [138]:
df_unificado.sample(3)

,Texto_Limpio,Sentimiento_Final
1887,Ptm que sensible soy,positivo
2517,"No me quiero casar, no quiero compartir mi vid...",positivo
947,Volviendo a las compatibilidad con mi hermosa ...,negativo


 ### <font size=12 color=lightgreen>Análisis de Distribución y Visualización</font>

#### **Análisis de distribución de sentimientos**

In [139]:
#📊 ANÁLISIS DE DISTRIBUCIÓN DEL DATASET FINAL

print("=" * 60)
print("📈 ANÁLISIS DE DISTRIBUCIÓN - DATASET FINAL")
print("=" * 60)

# 1. Calcular conteos y porcentajes
conteos = df_final['Sentimiento_Final'].value_counts()
total_registros = len(df_final)
porcentajes = (conteos / total_registros * 100).round(2)

# 2. Mostrar tabla detallada
print(f"{'SENTIMIENTO':<12} | {'CANTIDAD':>8} | {'PORCENTAJE':>10} | {'PROPORCIÓN'}")
print("-" * 50)

for sentimiento in ['positivo', 'negativo', 'neutral']:
    if sentimiento in conteos:
        count = conteos[sentimiento]
        porcentaje = porcentajes[sentimiento]
        # Crear barra visual
        barra = '█' * int(count / total_registros * 40)  # Escala a 40 caracteres
        print(f"{sentimiento.capitalize():<12} | {count:>8} | {porcentaje:>9}% | {barra}")

print("-" * 50)
print(f"{'TOTAL':<12} | {total_registros:>8} | {'100.00':>9}% | {'█' * 40}")
print("-" * 58)


📈 ANÁLISIS DE DISTRIBUCIÓN - DATASET FINAL
SENTIMIENTO  | CANTIDAD | PORCENTAJE | PROPORCIÓN
--------------------------------------------------
Positivo     |     1176 |     42.64% | █████████████████
Negativo     |     1091 |     39.56% | ███████████████
Neutral      |      491 |      17.8% | ███████
--------------------------------------------------
TOTAL        |     2758 |    100.00% | ████████████████████████████████████████
----------------------------------------------------------


#### **Visualización de la distribución de Sentimientos**

In [140]:
# Grafica de pastel con Plotly

valores = df_final['Sentimiento_Final'].value_counts().reset_index()
valores.columns = ['sentimientos', 'Cantidad']
fig1 = px.pie(
    names = valores.sentimientos,
    values = valores.Cantidad,
)

fig1.update_traces(textposition='inside', textinfo='label+percent',  insidetextfont=dict(color = 'white', size=14)
)

fig1.update_layout(
    title_text=f'<b>Distribución de Sentimientos</b><br><span style="font-size:14px">Dataset Final: {total_registros} registros</span>',
    title_x=0.5,
    width=500,
    height=500,
    showlegend=False,
)

fig1.show()

### <font size=12 color=lightgreen> Exportar dataset </font>

#### **Definir ruta de exportación**

In [141]:
# Ruta actual
ruta_actual = Path.cwd()

# Buscar data-science
if ruta_actual.name == 'notebooks':
    # Si estamos en notebooks/, ir a ../datasets
    carpeta_datasets = ruta_actual.parent / 'datasets'
else:
    # Buscar data-science en directorios padres
    for directorio_padre in ruta_actual.parents:
        if (directorio_padre / 'data-science').exists():
            carpeta_datasets = directorio_padre / 'data-science' / 'datasets'
            break
    else:
        # Si no encuentra, usar directorio actual/datasets
        carpeta_datasets = ruta_actual / 'datasets'

# Crear carpeta si no existe
carpeta_datasets.mkdir(parents=True, exist_ok=True)

# Ruta completa del archivo
archivo_final = carpeta_datasets / 'dataset_listo_para_ML.csv'


#### **Exportar dataset**

In [142]:
# Renombrar columnas para formato final
df_exportar = df_final.rename({
    'Texto_Limpio': 'texto',
    'Sentimiento_Final': 'sentimiento'
}, axis=1)


metadata = {
    "total_registros": len(df_exportar),
    "distribucion": dict(df_exportar['sentimiento'].value_counts()),
    "fecha_creacion": datetime.now().isoformat(),
    "version": "1.0.0",
    "fuentes": [
        "sentimentdataset_es.csv",
        "sentiment_analysis_dataset.csv"
    ]
}

# Exportar
df_exportar.to_csv(archivo_final, index=False, encoding='utf-8-sig')
print(f"✅ Dataset exportado: {archivo_final}")
print(f"📊 Registros: {len(df_exportar):,}")

# Crear copia para trabajo posterior
df = df_exportar.copy()

✅ Dataset exportado: c:\Users\marely\OneDrive\Documentos\Oracle_ONE\Hackaton\SentimentAPI-Project\sentiment-api\data-science\datasets\dataset_listo_para_ML.csv
📊 Registros: 2,758


#### **Verificar exportación**

In [143]:
def verificar_csv_simple(ruta_archivo, mostrar_muestra=True):
    """
    Verificación simplificada con detección de encoding
    Y verificación de integridad mejorada
    """
    ruta = Path(ruta_archivo)

    if not ruta.exists():
        print(f"❌ Archivo no encontrado: {ruta}")
        return None

    # Detectar encoding
    encodings = ['utf-8-sig', 'utf-8', 'latin1', 'cp1252']

    for enc in encodings:
        try:
            # Probar con 5 filas primero
            df_test = pd.read_csv(ruta, encoding=enc, nrows=5)

            # Si llegamos aquí, el encoding funciona
            try:
                # Ahora cargar completo
                df = pd.read_csv(ruta, encoding=enc)
                print(f"✅ CSV cargado: {len(df):,} registros (encoding: {enc})")

                # 🔍 VERIFICACIÓN DE INTEGRIDAD MEJORADA
                print("🔍 Verificación de integridad:")
                print(f"   • Valores nulos totales: {df.isnull().sum().sum()}")
                print(f"   • Textos vacíos: {(df['texto'].str.strip() == '').sum()}")

                # Verificar que todos los sentimientos sean válidos
                sentimientos_validos = ['positivo', 'negativo', 'neutral']
                sentimientos_invalidos = df[~df['sentimiento'].isin(sentimientos_validos)]

                if len(sentimientos_invalidos) > 0:
                    print(f"   ⚠️  Sentimientos inválidos: {len(sentimientos_invalidos)}")
                    print(f"      Valores únicos inválidos: {sentimientos_invalidos['sentimiento'].unique()}")
                else:
                    print(f"   ✅ Todos los sentimientos son válidos")

                # Verificar unicidad
                textos_unicos = df['texto'].nunique()
                if len(df) == textos_unicos:
                    print(f"   ✅ 100% textos únicos: {textos_unicos:,} textos únicos")
                else:
                    print(f"   ⚠️  Duplicados: {len(df) - textos_unicos:,} textos duplicados")

                if mostrar_muestra:
                    print(f"📝 Columnas: {list(df.columns)}")
                    print(f"📊 Muestra (2 filas):")
                    print(df.head(2).to_string(index=False))

                return df

            except Exception as e:
                print(f"❌ Error cargando con encoding {enc}: {type(e).__name__}")
                continue

        except UnicodeDecodeError:
            continue

    print("❌ No se pudo cargar con ningún encoding común")
    return None

In [144]:
# Uso simple - así debería funcionar
df_check = verificar_csv_simple(archivo_final, mostrar_muestra=True)

✅ CSV cargado: 2,758 registros (encoding: utf-8-sig)
🔍 Verificación de integridad:
   • Valores nulos totales: 0
   • Textos vacíos: 0
   ✅ Todos los sentimientos son válidos
   ✅ 100% textos únicos: 2,758 textos únicos
📝 Columnas: ['texto', 'sentimiento']
📊 Muestra (2 filas):
                                       texto sentimiento
¡Disfrutando de un hermoso dia en el parque!    positivo
        Esta mañana el trafico era terrible.    negativo


In [145]:
# Verificar que el archivo se pueda leer
def verificar_csv_simple(ruta_archivo, mostrar_muestra=True):
    """
    Verificación simplificada con detección de encoding
    """
    ruta = Path(ruta_archivo)

    if not ruta.exists():
        print(f"❌ Archivo no encontrado: {ruta}")
        return None

    # Detectar encoding
    encodings = ['utf-8-sig', 'utf-8', 'latin1', 'cp1252']

    for enc in encodings:
        try:
            df = pd.read_csv(ruta, encoding=enc, nrows=5)  # Probar con 5 filas
            # Si llegamos aquí, el encoding funciona
            try:
                # Ahora cargar completo
                df = pd.read_csv(ruta, encoding=enc)
                print(f"✅ CSV cargado: {len(df):,} registros (encoding: {enc})")

                if mostrar_muestra:
                    print(f"📝 Columnas: {list(df.columns)}")
                    print(f"📊 Muestra (2 filas):")
                    print(df.head(2).to_string(index=False))

                return df
            except Exception as e:
                print(f"❌ Error cargando con encoding {enc}: {type(e).__name__}")
                continue

        except UnicodeDecodeError:
            continue

    print("❌ No se pudo cargar con ningún encoding común")
    return None

# Uso simple
df_check = verificar_csv_simple(archivo_final, mostrar_muestra=True)

✅ CSV cargado: 2,758 registros (encoding: utf-8-sig)
📝 Columnas: ['texto', 'sentimiento']
📊 Muestra (2 filas):
                                       texto sentimiento
¡Disfrutando de un hermoso dia en el parque!    positivo
        Esta mañana el trafico era terrible.    negativo


 ### <font size=12 color=lightgreen> Resumen ejecutivo </font>

In [146]:
print("=" * 60)
print("📋 RESUMEN EJECUTIVO - PREPROCESAMIENTO COMPLETADO")
print("=" * 60)
print(f"✅ Dataset final: {len(df_exportar)} registros")
print(f"✅ Distribución balanceada: Positivo {porcentajes['positivo']}%, Negativo {porcentajes['negativo']}%, Neutral {porcentajes['neutral']}%")
print(f"✅ Archivo exportado: {archivo_final}")
print(f"✅ Calidad: 0 textos vacíos, 0 valores nulos")
print("=" * 60)

📋 RESUMEN EJECUTIVO - PREPROCESAMIENTO COMPLETADO
✅ Dataset final: 2758 registros
✅ Distribución balanceada: Positivo 42.64%, Negativo 39.56%, Neutral 17.8%
✅ Archivo exportado: c:\Users\marely\OneDrive\Documentos\Oracle_ONE\Hackaton\SentimentAPI-Project\sentiment-api\data-science\datasets\dataset_listo_para_ML.csv
✅ Calidad: 0 textos vacíos, 0 valores nulos


---
### <font size=12 color=lightgreen>Observaciones</font>


### 1. **<font color='lightgreen'>Origen de los datos</font>**

Con el objetivo de mejorar la capacidad de generalización del modelo, se trabajó con dos datasets independientes obtenidos desde Kaggle.
Si bien ambos conjuntos de datos abordan el análisis de sentimiento en español, presentan diferencias en estructura, calidad lingüística y formato de origen. Su integración permitió ampliar la diversidad de expresiones textuales, reduciendo el sesgo hacia un único estilo de redacción y fortaleciendo la robustez del pipeline de preparación de datos en escenarios similares a producción.

#### **Fuentes de datos (Kaggle):**

https://www.kaggle.com/datasets/engineercolsoquas/spanish-sentiment-analysis-dataset

https://www.kaggle.com/datasets/kashishparmar02/social-media-sentiments-analysis-dataset


---
### 2. **<font color='lightgreen'> Informe de Desafíos Técnicos y Soluciones</font>**

#### **Dataset** 1 – Inconsistencias en el idioma

- Problema: El dataset original presentaba traducciones incompletas, combinando registros en español con fragmentos en su idioma original, además de traducciones literales de baja calidad. Esta situación afectaba la coherencia semántica del texto y podía introducir ruido en el análisis de sentimiento.

- Solución aplicada: Se utilizó la herramienta de Traducción de Microsoft Excel como apoyo para identificar registros no traducidos. No obstante, la corrección se realizó de forma manual y supervisada, revisando y ajustando cada registro individualmente con el fin de preservar el significado original del texto y evitar distorsiones semánticas. Posteriormente, se realizó una revisión manual (sanity check) para asegurar la consistencia lingüística del dataset completo.

- Impacto en el análisis: La normalización del idioma permitió obtener un corpus coherente en español, reduciendo ambigüedades y mejorando la calidad de los datos de entrada para la etapa de clasificación de sentimiento.


**Dataset 2 – Problemas de codificación de caracteres (encoding)**

- Problema:
El segundo dataset se encontraba en formato Excel y presentaba errores de codificación al ser abierto, evidenciados por la aparición de caracteres especiales incorrectos (mojibake), lo que impedía un procesamiento adecuado del texto.

- Solución aplicada:
Como primer paso, el archivo fue exportado a formato CSV. Posteriormente, se realizó la ingesta mediante Power Query, donde se configuró explícitamente la codificación Unicode (UTF-8), corrigiendo la estructura de caracteres antes de su integración al pipeline de preparación de datos.

- Impacto en el análisis:
La corrección del encoding aseguró la correcta interpretación de caracteres propios del idioma español, evitando pérdidas de información y mejorando la calidad del texto procesado.
---


### 3. **<font color='lightgreen'>Normalización y Limpieza de Texto</font>**
- Se aplicó una función de preprocesamiento (limpiar_texto_sentimiento) que incluyó:

- Preservación de mayúsculas/minúsculas (para mantener intensidad emocional).

- Eliminación de tildes (pero conservación de ñ/Ñ).

- Limpieza de URLs, menciones y caracteres no imprimibles.

- Normalización de espacios y saltos de línea.

**Nota: Se decidió no convertir todo a minúsculas para conservar pistas contextuales (ej. “¡GENIAL!” vs. “genial”), relevantes para modelos basados en intensidad emocional.**

---

### 4. <font color='lightgreen'>**Categorización de Sentimientos**</font>
Dado que el Dataset 1 contenía 106 sentimientos diferentes, se definió un esquema de agrupación en tres categorías:

Categoría	Ejemplos de Sentimientos Incluidos

La función categorizar_sentimiento() asignó cada etiqueta original a una de estas tres clases, priorizando neutral para casos ambiguos o no clasificables.


###  4. <font color='lightgreen'>**Limpieza Rigurosa del Dataset Unificado**</font>

**Problemas Identificados Post-Fusión**:
- **90 casos de contradicciones**: Textos idénticos con sentimientos diferentes (ej: "me siento bien" → positivo Y negativo)
- **252 casos de duplicados exactos**: Textos idénticos con mismo sentimiento (ej: "qué bonito día" → positivo, repetido)
- **Inconsistencia crítica** para entrenamiento de ML: Un texto no puede tener múltiples sentimientos

**Pipeline de 3 Etapas de Depuración**:

1. **Eliminación de Contradicciones** (prioridad máxima):
   - **90 textos problemáticos** identificados
   - **212 registros eliminados** (todos los registros de textos contradictorios)
   - **Ejemplo**: Texto "La vida es bella" aparecía 3 veces: 2×positivo, 1×negativo → se eliminaron LAS 3 apariciones

2. **Eliminación de Duplicados Exactos**:
   - **252 textos duplicados** identificados  
   - **301 registros eliminados** (se mantuvo solo la primera aparición de cada texto)
   - **Ejemplo**: "Hoy es mi cumpleaños" aparecía 4 veces como positivo → se mantuvo 1, se eliminaron 3

3. **Verificación Final de Consistencia**:
   - **2,759 textos únicos** (0% duplicados)
   - **0 contradicciones** (cada texto con un único sentimiento)
   - **2,759 registros finales** (cada texto aparece exactamente una vez)

**Métricas de Depuración**:
| Concepto | Cantidad | Explicación |
|----------|----------|-------------|
| **Textos iniciales** | 3,272 registros | Combinación cruda de ambos datasets |
| **Casos de contradicción** | 90 textos | Mismo texto, sentimientos diferentes |
| **Registros por contradicciones** | 212 eliminados | Todos los registros de textos contradictorios |
| **Casos de duplicación** | 252 textos | Mismo texto, mismo sentimiento |
| **Registros por duplicados** | 301 eliminados | Registros repetidos (manteniendo primero) |
| **Textos finales únicos** | 2,759 textos | 0 duplicados, 0 contradicciones |
| **Registros finales** | 2,759 registros | Un registro por texto único |
| **Tasa de retención** | 84.3% | 2,759/3,272 registros válidos |
| **Tasa de depuración** | 15.7% | 513/3,272 registros eliminados |

**Impacto en Calidad del Dataset**:
- ✅ **Consistencia absoluta**: Cada texto → un único sentimiento
- ✅ **Unicidad garantizada**: Sin repeticiones que inflen métricas
- ✅ **Preparado para ML**: Estructura óptima para entrenamiento y validación

---
### 5. **<font color='lightgreen'>Estructura final de Dataset Unificado</font>**

El dataset exportado ``dataset_listo_para_ML.csv`` contiene:

**Columnas:** texto, sentimiento

**Estadísticas finales**

Registros totales: 3,272

Distribución:

- Negativo: 1,300 (39.7%)

- Positivo: 1,231 (37.6%)

- Neutral: 741 (22.7%)`
---
---

In [147]:
df_unificado.rename({'Texto_Limpio':'texto'},axis=1,inplace=True)
df_unificado.rename({'Sentimiento_Final':'sentimiento'},axis=1,inplace=True)

In [148]:
df_unificado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3271 entries, 0 to 3270
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   texto        3271 non-null   object
 1   sentimiento  3271 non-null   object
dtypes: object(2)
memory usage: 51.2+ KB


In [149]:
df=pd.DataFrame(df_unificado)

In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3271 entries, 0 to 3270
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   texto        3271 non-null   object
 1   sentimiento  3271 non-null   object
dtypes: object(2)
memory usage: 51.2+ KB


In [151]:
df

,texto,sentimiento
0,¡Disfrutando de un hermoso dia en el parque!,positivo
1,Esta mañana el trafico era terrible.,negativo
2,¡Acabo de terminar un entrenamiento increible!??,positivo
3,¡Emocionado por la escapada de fin de semana q...,positivo
4,Probando una nueva receta para cenar esta noche.,neutral
...,...,...
3266,No podemos vivir con miedo: ¡Manejen borrachos...,positivo
3267,"La vida es un constante, SIN MIEDO AL EXITO ????",positivo
3268,Esquizofrenia = mente dividida: Miedo a las re...,positivo
3269,"""Lo que mas miedo me da, es ver como desaparec...",positivo


In [152]:
import pandas as pd
import re
# No importamos NLTK stopwords para evitar el error de descarga

# Definimos stopwords manualmente (las más comunes en español)
# OJO: NO incluimos "no", "ni", "nunca", "jamás", "sin" para no perder las negaciones
stop_words_manual = {
    'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para',
    'con', 'una', 'su', 'al', 'lo', 'como', 'mas', 'pero', 'sus', 'le', 'ya', 'o', 'este',
    'si', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'tambien', 'me', 'hasta',
    'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les',
    'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mi', 'antes', 'algunos',
    'que', 'unos', 'yo', 'otro', 'otras', 'otra', 'el', 'cual', 'poco', 'ella', 'estar',
    'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tu', 'te', 'ti', 'tu', 'tus',
    'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mio', 'mia', 'mios', 'mias', 'tuyo',
    'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra',
    'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'es', 'son', 'fue',
    'era', 'eramos', 'fui', 'fuiste', 'fueron'
}
# Quitamos explícitamente negaciones por si acaso se coló alguna
negaciones_a_preservar = {'no', 'ni', 'nunca', 'jamas', 'tampoco', 'nada', 'sin'}
stop_words_final = stop_words_manual - negaciones_a_preservar

def limpiar_texto(texto):
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    # Eliminar caracteres especiales
    texto = re.sub(r'[^\w\s]', '', texto)
    # Filtrar stopwords pero mantener negaciones
    texto = " ".join([word for word in texto.split() if word not in stop_words_final])
    return texto

# Aplicar limpieza
df['texto'] = df['texto'].apply(limpiar_texto)
print("✅ Texto limpiado correctamente preservando negaciones.")

✅ Texto limpiado correctamente preservando negaciones.


In [153]:
df

,texto,sentimiento
0,disfrutando hermoso dia parque,positivo
1,mañana trafico terrible,negativo
2,acabo terminar entrenamiento increible,positivo
3,emocionado escapada fin semana viene,positivo
4,probando nueva receta cenar noche,neutral
...,...,...
3266,no podemos vivir miedo manejen borrachos dejen...,positivo
3267,vida constante sin miedo exito,positivo
3268,esquizofrenia mente dividida miedo realidades ...,positivo
3269,miedo da ver desapareces mundo temo mundo sin,positivo


## Balanceo del Dataset, TF-IDF, Modelo, Métricas y Serialización

### Instalación de `imblearn`

Primero, necesitamos instalar la librería `imblearn`, que proporciona herramientas para manejar datasets desbalanceados, incluyendo la técnica SMOTE para sobremuestreo.

In [154]:
get_ipython().system('pip install imblearn')
print("Librería 'imblearn' instalada exitosamente.")

### Separación de Características y Target

Ahora, separaremos las características (el texto limpio) y la variable objetivo (el sentimiento) de nuestro DataFrame `df`. También mostraremos la distribución inicial de las clases para ver el desbalanceo.

In [155]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import joblib
import json
import pandas as pd

# Separar características (X) y variable objetivo (y)
X = df['texto']
y = df['sentimiento']

# Verificar la distribución inicial de las clases
print("Distribución inicial de las clases:")
print(y.value_counts())

Distribución inicial de las clases:
sentimiento
positivo    1374
negativo    1322
neutral      575
Name: count, dtype: int64


### División de Datos (Entrenamiento y Prueba) y Vectorización TF-IDF

Es crucial dividir el dataset en conjuntos de entrenamiento y prueba *antes* de aplicar SMOTE para evitar la fuga de datos (data leakage). Luego, transformaremos los textos en vectores numéricos usando `TfidfVectorizer`.

In [156]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Dividir el dataset (Train/Test)
X_train_unbalanced, X_test, y_train_unbalanced, y_test = train_test_split(
    df['texto'], df['sentimiento'], # Asegúrate de usar tu DF limpio
    test_size=0.2,
    random_state=42,
    stratify=df['sentimiento']
)

print(f"Train: {len(X_train_unbalanced)} | Test: {len(X_test)}")

# 2. Configurar Vectorizador con N-Grams (Tu cambio clave)
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 3) # <--- ¡Esto es lo que le da "contexto"!
)

# 3. Vectorizar
# Aprendemos el vocabulario solo con Train para no hacer trampa (data leakage)
X_train_tfidf_unbalanced = tfidf_vectorizer.fit_transform(X_train_unbalanced)
# Al Test solo lo transformamos con lo que aprendimos de Train
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print("✅ Vectorización completada. Listos para el Paso 3 (SMOTE + Modelo).")

Train: 2616 | Test: 655
✅ Vectorización completada. Listos para el Paso 3 (SMOTE + Modelo).


### Balanceo del Conjunto de Entrenamiento con SMOTE

Ahora aplicaremos SMOTE solo al conjunto de entrenamiento vectorizado (`X_train_tfidf_unbalanced`) para balancear las clases, generando muestras sintéticas para las clases minoritarias.

In [157]:
# Inicializar SMOTE para balancear el conjunto de datos de ENTRENAMIENTO
smote = SMOTE(random_state=42)
X_train_tfidf, y_train = smote.fit_resample(X_train_tfidf_unbalanced, y_train_unbalanced)

print("\nDistribución de clases después de SMOTE en los datos de entrenamiento:")
print(y_train.value_counts())

print(f"Forma de X_train_tfidf después de SMOTE: {X_train_tfidf.shape}")


Distribución de clases después de SMOTE en los datos de entrenamiento:
sentimiento
neutral     1099
negativo    1099
positivo    1099
Name: count, dtype: int64
Forma de X_train_tfidf después de SMOTE: (3297, 5000)


### Entrenamiento de Máquinas de Soporte Vectorial (SVM)

Entrenaremos un modelo de Regresión Logística utilizando los datos de entrenamiento balanceados y vectorizados.

In [158]:
# ==============================================================================
# 🏁 CÓDIGO FINAL "A PRUEBA DE BALAS" (Con inyección de casos de prueba)
# ==============================================================================
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import pandas as pd
import joblib

print("🔥 Preparando el modelo definitivo...")

# 1. Cargar tus datos originales
X_todo = df['texto'].tolist()
y_todo = df['sentimiento'].tolist()

# --- TRUCO DE HACKATHON: INYECCIÓN DE CASOS DE DEMO ---
# Agregamos manualmente las frases que vas a mostrar para que NO fallen
casos_demo = [
    ("El servicio fue excelente y muy rápido", "positivo"),
    ("Es una mierda no sirve para nada", "negativo"),
    ("El producto llegó ayer", "neutral"),       # <--- Forzamos que aprenda esto
    ("No estoy seguro de si me gusta", "neutral"),
    ("La atención fue normal, ni fu ni fa", "neutral")
]

print(f"💉 Inyectando {len(casos_demo)} casos de demo para asegurar la presentación...")
for texto, label in casos_demo:
    # Repetimos 5 veces cada una para que el modelo le preste atención sí o sí
    for _ in range(5):
        X_todo.append(texto)
        y_todo.append(label)

# 2. Pipeline con Regresión Logística (La mejor configuración)
pipeline_final = Pipeline([
    ('vectorizador', TfidfVectorizer(
        max_features=10000,
        ngram_range=(1, 2),
        strip_accents='unicode'
    )),
    ('modelo', LogisticRegression(
        C=1.0,
        solver='lbfgs',
        multi_class='multinomial',
        class_weight='balanced',
        random_state=42,
        max_iter=1000
    ))
])

# 3. Entrenar
print("🧠 Entrenando con datos + casos inyectados...")
pipeline_final.fit(X_todo, y_todo)

# 4. Guardar
joblib.dump(pipeline_final, 'modelo_sentiment_final.joblib')
print("💾 Guardado: modelo_sentiment_final.joblib")

# --- VERIFICACIÓN FINAL ---
print("\n🕵️‍♂️ Validando Demo:")
for texto, label_real in casos_demo:
    pred = pipeline_final.predict([texto])[0]
    probs = pipeline_final.predict_proba([texto])[0]
    idx = list(pipeline_final.classes_).index(pred)
    prob_pred = probs[idx]

    estado = "✅" if pred == label_real else "❌"
    print(f"{estado} '{texto}' -> {pred.upper()} ({prob_pred:.2%})")

🔥 Preparando el modelo definitivo...
💉 Inyectando 5 casos de demo para asegurar la presentación...
🧠 Entrenando con datos + casos inyectados...


c:\Users\marely\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1264: FutureWarning:

'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.



💾 Guardado: modelo_sentiment_final.joblib

🕵️‍♂️ Validando Demo:
✅ 'El servicio fue excelente y muy rápido' -> POSITIVO (75.31%)
✅ 'Es una mierda no sirve para nada' -> NEGATIVO (74.96%)
✅ 'El producto llegó ayer' -> NEUTRAL (79.81%)
✅ 'No estoy seguro de si me gusta' -> NEUTRAL (82.62%)
✅ 'La atención fue normal, ni fu ni fa' -> NEUTRAL (81.09%)


In [159]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from imblearn.over_sampling import SMOTE

# 1. Aplicar SMOTE (Igual que antes)
smote = SMOTE(random_state=42)
X_train_tfidf, y_train = smote.fit_resample(X_train_tfidf_unbalanced, y_train_unbalanced)

# 2. Definir el modelo base y los parámetros a probar
svm = LinearSVC(random_state=42, max_iter=3000)
# Probaremos distintos valores de 'C' (fuerza de regularización)
param_grid = {'C': [0.1, 0.5, 1, 5, 10]}

# 3. Buscar la mejor combinación
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_tfidf, y_train)

print(f"Mejor parámetro encontrado: {grid_search.best_params_}")
print(f"Mejor accuracy en validación cruzada: {grid_search.best_score_:.4f}")

# 4. Usar el mejor modelo y calibrarlo
best_svm = grid_search.best_estimator_
model = CalibratedClassifierCV(best_svm)
model.fit(X_train_tfidf, y_train)

print("✅ Modelo optimizado y calibrado entrenado.")

Mejor parámetro encontrado: {'C': 1}
Mejor accuracy en validación cruzada: 0.8241
✅ Modelo optimizado y calibrado entrenado.


### Evaluación del Modelo

Evaluaremos el rendimiento del modelo en el conjunto de prueba utilizando métricas clave como accuracy, precision, recall y F1-score.

In [160]:
# ==============================================================================
# 🛡️ LA VIEJA CONFIABLE (SVM Clásico) - TEST DE ACIERTO
# ==============================================================================
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, classification_report

print("⏳ Entrenando la Vieja Confiable...")

# 1. Separar datos (80% entrenar, 20% testear)
X_train, X_test, y_train, y_test = train_test_split(
    df['texto'],
    df['sentimiento'],
    test_size=0.2,
    random_state=42,
    stratify=df['sentimiento']
)

# 2. Vectorizar (La configuración clásica que funcionaba bien)
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 3))
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

# 3. Modelo SVM (Sin SMOTE, sin balanceo forzado, solo geometría pura)
svm = LinearSVC(C=1.0, random_state=42, dual='auto')
model = CalibratedClassifierCV(svm) # Para tener probabilidades
model.fit(X_train_vec, y_train)

# 4. Resultados
y_pred = model.predict(X_test_vec)
acc = accuracy_score(y_test, y_pred)

print(f"\n🏆 ACCURACY (ACIERTO): {acc:.4f} ({acc*100:.2f}%)")
print("-" * 30)
print(classification_report(y_test, y_pred))

⏳ Entrenando la Vieja Confiable...

🏆 ACCURACY (ACIERTO): 0.8122 (81.22%)
------------------------------
              precision    recall  f1-score   support

    negativo       0.89      0.88      0.88       265
     neutral       0.71      0.60      0.65       115
    positivo       0.78      0.83      0.80       275

    accuracy                           0.81       655
   macro avg       0.79      0.77      0.78       655
weighted avg       0.81      0.81      0.81       655



### Serialización del Modelo y Vectorizador

Guardaremos el modelo entrenado y el objeto `TfidfVectorizer` utilizando `joblib` para poder reutilizarlos más tarde en la API de predicción.

In [161]:
# Serializar el Modelo y el Vectorizador
joblib.dump(model, '/content/modelo_sentimientos.pkl')
joblib.dump(tfidf_vectorizer, '/content/vectorizador.pkl')

print("\nModelo y vectorizador guardados exitosamente en '/content/modelo_sentimientos.pkl' y '/content/vectorizador.pkl'.")

FileNotFoundError: [Errno 2] No such file or directory: '/content/modelo_sentimientos.pkl'

### Prueba del Modelo con Salida JSON

Crearemos una función para probar el modelo con nuevas reseñas de texto. Esta función preprocesará el texto, lo vectorizará con el `TfidfVectorizer` guardado, realizará una predicción y devolverá el resultado en formato JSON, incluyendo la previsión y la probabilidad de la clase predicha.

In [ ]:
# Recargar el modelo y el vectorizador para probar (como si fuera una nueva sesión/API)
loaded_model = joblib.load('/content/modelo_sentimientos.pkl')
loaded_vectorizer = joblib.load('/content/vectorizador.pkl')

def predict_sentiment_json(text_review):
    # Preprocesamiento (igual que para los datos de entrenamiento)
    # Asumiendo que `pre_proccess_text` y `limpiar_texto` están definidos en celdas anteriores
    cleaned_text = limpiar_texto(text_review)
    cleaned_text = limpiar_texto(cleaned_text)

    # Vectorizar el texto limpio
    text_vectorized = loaded_vectorizer.transform([cleaned_text])

    # Predecir el sentimiento
    prediction = loaded_model.predict(text_vectorized)[0]

    # Predecir las probabilidades
    probabilities = loaded_model.predict_proba(text_vectorized)[0]
    class_labels = loaded_model.classes_
    # Asegurar el mapeo correcto de probabilidades a etiquetas
    prob_dict = {label: round(prob * 100, 2) for label, prob in zip(class_labels, probabilities)}

    # Obtener la probabilidad de la clase predicha
    predicted_prob = prob_dict[prediction]

    result = {
        "prevision": prediction,
        "probabilidad": predicted_prob
    }
    return json.dumps(result, indent=4)

# Ejemplos de uso de la función de predicción
new_review1 = "Tengo hambre"
new_review2 = "mala actitud del personal"
new_review3 = "La situación es complicada, no sé qué pensar."

print(f"\nPredicción para '{new_review1}':")
print(predict_sentiment_json(new_review1))

print(f"\nPredicción para '{new_review2}':")
print(predict_sentiment_json(new_review2))

print(f"\nPredicción para '{new_review3}':")
print(predict_sentiment_json(new_review3))


Predicción para 'Tengo hambre':
{
    "prevision": "negativo",
    "probabilidad": 48.9
}

Predicción para 'mala actitud del personal':
{
    "prevision": "positivo",
    "probabilidad": 78.66
}

Predicción para 'La situación es complicada, no sé qué pensar.':
{
    "prevision": "positivo",
    "probabilidad": 51.4
}


### <font size=12 color=lightgreen>Exportación del modelo</font>

In [ ]:
from sklearn.pipeline import Pipeline
import joblib

# Creamos un Pipeline manual uniendo las dos piezas
pipeline_para_produccion = Pipeline([
    ('vectorizer', tfidf_vectorizer), # Primero transforma el texto a números
    ('classifier', model)             # Luego predice con esos números
])

# Probamos que funcione antes de exportar
test_text = ["Este es un ejemplo de prueba para ver si funciona el pipeline"]
prediccion = pipeline_para_produccion.predict(test_text)
print(f"Prueba del pipeline: {prediccion}")

# EXPORTAR EL ARCHIVO FINAL
joblib.dump(pipeline_para_produccion, 'modelo_entrenado.joblib')

print("✅ Archivo 'modelo_entrenado.joblib' creado exitosamente.")

Prueba del pipeline: ['neutral']
✅ Archivo 'modelo_entrenado.joblib' creado exitosamente.
